In [6]:
#installing BS and requests
!pip install BeautifulSoup4
!pip install requests

In [8]:
# importing libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

# defining source
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'html')

# scraping the page
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    if (index == 0):
        columns = section
    else:
        data.append(section)

# 1 creating dataframe 
canada_df = pd.DataFrame(data = data,columns = columns)
canada_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
# 2 dropping NA boroughs
canada_df = canada_df[canada_df.Borough != 'Not assigned'].reset_index(drop=True)
canada_df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
# 3 grouping by postal code and boroughs
canada_df = canada_df.groupby(['Postal Code','Borough'], as_index=False).agg(lambda x: ','.join(x))
canada_df.head(86)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
81,M6N,York,"Runnymede, The Junction North"
82,M6P,West Toronto,"High Park, The Junction South"
83,M6R,West Toronto,"Parkdale, Roncesvalles"
84,M6S,West Toronto,"Runnymede, Swansea"


In [10]:
# 4 replacing NA with Boroughs
canada_df['Neighborhood'].replace("Not assigned", canada_df["Borough"],inplace=True)
canada_df.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [12]:
canada_df.shape

(103, 3)

<b> Part 2 Start </b>

In [13]:
# get coordinates dataframe
!wget -q -O "canada_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
coors = pd.read_csv('canada_coordinates.csv')

Coordinates downloaded!


In [14]:
print(coors.shape)
coors.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
#merging dataframes 
canada_df_temp = canada_df.set_index('Postal Code')
coors_temp = coors.set_index('Postal Code')
canada_df_coors = pd.concat([canada_df_temp, coors_temp], axis=1, join='inner')

canada_df_coors.index.name = 'Postal Code'
canada_df_coors.reset_index(inplace=True)

print(canada_df_coors.shape)
canada_df_coors.head(12)

(103, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


<b> Part 3 start </b>

In [22]:
# defining coordinates

address = 'Toronto, Ontario'

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="tl-toronto-neigh")

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

Solving environment: done

# All requested packages already installed.

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [25]:
# importing folium and creating Toronto map
import folium

map_canada = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(canada_df_coors['Latitude'], canada_df_coors['Longitude'], canada_df_coors['Postal Code'], canada_df_coors['Borough'], canada_df_coors['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)
    
map_canada

In [26]:
# limiting boroughs
canada_boroughs = ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']
canada_central_df = canada_df_coors[canada_df_coors['Borough'].isin(canada_boroughs)].reset_index(drop=True)
print(canada_central_df.shape)
canada_central_df.head()

(39, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [27]:
# creating updated map
map_canada = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, long, post, borough, neigh in zip(canada_central_df['Latitude'], canada_central_df['Longitude'], canada_central_df['Postal Code'], canada_central_df['Borough'], canada_central_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)
    
map_canada


<b> Exploring boroughs via foursquare </b>

In [29]:
# defining credentials
CLIENT_ID = 'W44RWKJ1W0B3DGCX0FNHUK1X4BN0HNRJAGSYD2B0HMZPYK4Z' 
CLIENT_SECRET = 'S2FT222VEEMFVOGHDDEZKUYSU1U5VYVHPNUQYTFDHH5AEDK5' 
VERSION = '20190330' 

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: W44RWKJ1W0B3DGCX0FNHUK1X4BN0HNRJAGSYD2B0HMZPYK4Z
CLIENT_SECRET:S2FT222VEEMFVOGHDDEZKUYSU1U5VYVHPNUQYTFDHH5AEDK5


In [31]:
# limiting venues
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(canada_central_df['Latitude'], canada_central_df['Longitude'], canada_central_df['Postal Code'], canada_central_df['Borough'], canada_central_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [32]:
# creating venues dataframe
venues_df = pd.DataFrame(venues)
venues_df.columns = ['Postal Code', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(1612, 9)


,Postal Code,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [36]:
# grouping and counting venues
venues_df.groupby(['Postal Code', 'Borough', 'Neighborhood'])['VenueName'].count()


Postal Code  Borough           Neighborhood                                                                                              
M4E          East Toronto      The Beaches                                                                                                     4
M4K          East Toronto      The Danforth West, Riverdale                                                                                   42
M4L          East Toronto      India Bazaar, The Beaches West                                                                                 21
M4M          East Toronto      Studio District                                                                                                41
M4N          Central Toronto   Lawrence Park                                                                                                   3
M4P          Central Toronto   Davisville North                                                                                          

In [37]:
len(venues_df['VenueCategory'].unique())

240

In [38]:
# analyzing venues for city areas

# one hot encoding
canada_central_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column to dataframe
canada_central_onehot['Postal Code'] = venues_df['Postal Code'] 
canada_central_onehot['Borough'] = venues_df['Borough'] 
canada_central_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column
fixed_columns = list(canada_central_onehot.columns[-3:]) + list(canada_central_onehot.columns[:-3])
canada_central_onehot = canada_central_onehot[fixed_columns]

print(canada_central_onehot.shape)
canada_central_onehot.head()

(1612, 243)


,Postal Code,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West, Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
# calculating frequency for each category
canada_central_venues_freq = canada_central_onehot.groupby(['Postal Code', 'Borough', 'Neighborhoods']).mean().reset_index()
print(canada_central_venues_freq.shape)
canada_central_venues_freq.head()

(39, 243)


,Postal Code,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.02381
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000
3,M4M,East Toronto,Studio District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.02439,0.0,0.0,0.0,0.02439
4,M4N,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000


In [42]:
# displaying top 10 venues for each area
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['Postal Code', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = canada_central_venues_freq['Postal Code']
neighborhoods_venues_sorted['Borough'] = canada_central_venues_freq['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = canada_central_venues_freq['Neighborhoods']

for ind in np.arange(canada_central_venues_freq.shape[0]):
    row_categories = canada_central_venues_freq.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
neighborhoods_venues_sorted

,Postal Code,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Harbor / Marina,Sculpture Garden,Coffee Shop,Plane,Boat or Ferry,Bar,Airport Terminal,Airport Gate
31,M6H,West Toronto,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Middle Eastern Restaurant,Bank,Bar,Café,Pool,Supermarket,Music Venue,Grocery Store
32,M6J,West Toronto,"Little Portugal, Trinity",Bar,Coffee Shop,Asian Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Café,Men's Store,Yoga Studio,Juice Bar,Korean Restaurant
25,M5S,Downtown Toronto,"University of Toronto, Harbord",Café,Bakery,Bookstore,Bar,Italian Restaurant,Japanese Restaurant,Restaurant,Bank,Beer Bar,Beer Store
33,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Coffee Shop,Climbing Gym,Burrito Place,Stadium,Italian Restaurant,Intersection,Restaurant
3,M4M,East Toronto,Studio District,Café,Coffee Shop,American Restaurant,Brewery,Bakery,Gastropub,Yoga Studio,Cheese Shop,Ice Cream Shop,Bookstore
26,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",Café,Mexican Restaurant,Coffee Shop,Vietnamese Restaurant,Bakery,Bar,Grocery Store,Gaming Cafe,Vegetarian / Vegan Restaurant,Dessert Shop
34,M6P,West Toronto,"High Park, The Junction South",Café,Thai Restaurant,Mexican Restaurant,Music Venue,Flea Market,Bakery,Italian Restaurant,Cajun / Creole Restaurant,Speakeasy,Fast Food Restaurant
14,M5B,Downtown Toronto,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Restaurant,Italian Restaurant,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Tea Room
19,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Café,Hotel,Italian Restaurant,Fried Chicken Joint,Brewery,Restaurant,Scenic Lookout,Sporting Goods Shop


In [44]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# running k-means for clustering 
kclusters = 5

canada_central_venues_freq_clustering = canada_central_venues_freq.drop(['Postal Code', 'Borough', 'Neighborhoods'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(canada_central_venues_freq_clustering)

canada_central_clustered_df = canada_central_df
canada_central_clustered_df['Cluster'] = kmeans.labels_

canada_central_clustered_df = canada_central_clustered_df.join(neighborhoods_venues_sorted.drop(['Borough', 'Neighborhoods'], 1).set_index('Postal Code'), on='Postal Code')
canada_central_clustered_df.sort_values(['Cluster'] + freqColumns, inplace=True)
canada_central_clustered_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0,Park,Playground,Summer Camp,Restaurant,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
10,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Trail,Playground,Yoga Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run
27,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,1,Airport Service,Airport Lounge,Harbor / Marina,Sculpture Garden,Coffee Shop,Plane,Boat or Ferry,Bar,Airport Terminal,Airport Gate
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1,Bakery,Pharmacy,Middle Eastern Restaurant,Bank,Bar,Café,Pool,Supermarket,Music Venue,Grocery Store
32,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,1,Bar,Coffee Shop,Asian Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Café,Men's Store,Yoga Studio,Juice Bar,Korean Restaurant
25,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,1,Café,Bakery,Bookstore,Bar,Italian Restaurant,Japanese Restaurant,Restaurant,Bank,Beer Bar,Beer Store
33,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,1,Café,Breakfast Spot,Nightclub,Coffee Shop,Climbing Gym,Burrito Place,Stadium,Italian Restaurant,Intersection,Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,American Restaurant,Brewery,Bakery,Gastropub,Yoga Studio,Cheese Shop,Ice Cream Shop,Bookstore
26,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,1,Café,Mexican Restaurant,Coffee Shop,Vietnamese Restaurant,Bakery,Bar,Grocery Store,Gaming Cafe,Vegetarian / Vegan Restaurant,Dessert Shop


In [46]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# creating clusters map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# appending markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(canada_central_clustered_df['Latitude'], canada_central_clustered_df['Longitude'], canada_central_clustered_df['Postal Code'], canada_central_clustered_df['Borough'], canada_central_clustered_df['Neighborhood'], canada_central_clustered_df['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<b> Examination of cluster </b>

In [52]:
# cluster 1
canada_central_clustered_df.loc[canada_central_clustered_df['Cluster'] == 0, canada_central_clustered_df.columns[[1] + list(range(5, canada_central_clustered_df.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,0,Park,Playground,Summer Camp,Restaurant,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
4,Central Toronto,0,Park,Swim School,Bus Line,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
10,Downtown Toronto,0,Park,Trail,Playground,Yoga Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run


In [53]:
# cluster 2
canada_central_clustered_df.loc[canada_central_clustered_df['Cluster'] == 1, canada_central_clustered_df.columns[[1] + list(range(5, canada_central_clustered_df.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Downtown Toronto,1,Airport Service,Airport Lounge,Harbor / Marina,Sculpture Garden,Coffee Shop,Plane,Boat or Ferry,Bar,Airport Terminal,Airport Gate
31,West Toronto,1,Bakery,Pharmacy,Middle Eastern Restaurant,Bank,Bar,Café,Pool,Supermarket,Music Venue,Grocery Store
32,West Toronto,1,Bar,Coffee Shop,Asian Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Café,Men's Store,Yoga Studio,Juice Bar,Korean Restaurant
25,Downtown Toronto,1,Café,Bakery,Bookstore,Bar,Italian Restaurant,Japanese Restaurant,Restaurant,Bank,Beer Bar,Beer Store
33,West Toronto,1,Café,Breakfast Spot,Nightclub,Coffee Shop,Climbing Gym,Burrito Place,Stadium,Italian Restaurant,Intersection,Restaurant
3,East Toronto,1,Café,Coffee Shop,American Restaurant,Brewery,Bakery,Gastropub,Yoga Studio,Cheese Shop,Ice Cream Shop,Bookstore
26,Downtown Toronto,1,Café,Mexican Restaurant,Coffee Shop,Vietnamese Restaurant,Bakery,Bar,Grocery Store,Gaming Cafe,Vegetarian / Vegan Restaurant,Dessert Shop
34,West Toronto,1,Café,Thai Restaurant,Mexican Restaurant,Music Venue,Flea Market,Bakery,Italian Restaurant,Cajun / Creole Restaurant,Speakeasy,Fast Food Restaurant
14,Downtown Toronto,1,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Restaurant,Italian Restaurant,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Tea Room
19,Downtown Toronto,1,Coffee Shop,Aquarium,Café,Hotel,Italian Restaurant,Fried Chicken Joint,Brewery,Restaurant,Scenic Lookout,Sporting Goods Shop


In [54]:
# cluster 3
canada_central_clustered_df.loc[canada_central_clustered_df['Cluster'] == 2, canada_central_clustered_df.columns[[1] + list(range(5, canada_central_clustered_df.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Pool,Garden,Yoga Studio,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [55]:
# cluster 4
canada_central_clustered_df.loc[canada_central_clustered_df['Cluster'] == 3, canada_central_clustered_df.columns[[1] + list(range(5, canada_central_clustered_df.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,3,Trail,Neighborhood,Pub,Health Food Store,Yoga Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store


In [56]:
# cluster 5
canada_central_clustered_df.loc[canada_central_clustered_df['Cluster'] == 4, canada_central_clustered_df.columns[[1] + list(range(5, canada_central_clustered_df.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,4,Jewelry Store,Mexican Restaurant,Trail,Sushi Restaurant,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
